In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import zipfile
import os

In [ ]:
!pip install transformers timm torch torchvision pandas pillow

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import pandas as pd
import os
import numpy as np


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 59.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 45.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from PIL import Image, UnidentifiedImageError # Import UnidentifiedImageError
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import transforms
from transformers import AutoTokenizer

In [ ]:
import zipfile
import os

# Define paths
zip_path = "/content/drive/MyDrive/Multimodal_dataset_assignment3/Multimodal_dataset_assignment3/images.zip"  # Update with correct file name
extract_path = "/content/images"  # Destination folder

# Create extraction directory if it doesn't exist
os.makedirs(extract_path, exist_ok=True)

# Extract zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction Complete! Files are in:", extract_path)

Extraction Complete! Files are in: /content/images


In [ ]:
# Change these paths to match your dataset location
CSV_FILE_PATH = "/content/drive/MyDrive/Multimodal_dataset_assignment3/Multimodal_dataset_assignment3/labels.csv"  # Replace with actual path
IMG_DIR_PATH = "/content/images/images"  # Replace with actual path


In [ ]:
# Image Preprocessing
image_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load Pretrained Tokenizer (BERT/RoBERTa)
TEXT_MODEL = "bert-base-uncased"  # Change to "roberta-base" if using RoBERTa
tokenizer = AutoTokenizer.from_pretrained(TEXT_MODEL)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
class MemeDataset(Dataset):
    def __init__(self, csv_file, img_dir, tokenizer, transform=None, max_length=128):
        self.data = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.tokenizer = tokenizer
        self.transform = transform
        self.max_length = max_length

        self.label_mapping = {
            "not_funny": 0, "funny": 1, "very_funny": 2, "hilarious": 3,
            "not_sarcastic": 0, "general": 1, "twisted_meaning": 2, "very_twisted": 3,
            "not_offensive": 0, "slight": 1, "very_offensive": 2, "hateful_offensive": 3,
            "not_motivational": 0, "motivational": 1
        }

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        try:
            # Load Image
            img_name = os.path.join(self.img_dir, self.data.iloc[idx, 1])  # First column = image name
            image = Image.open(img_name).convert("RGB")
            if self.transform:
                image = self.transform(image)

            # Tokenize Text
            caption = self.data.iloc[idx, 3]  # Second column = caption

            # Ensure caption is a string
            caption = str(caption)

            encoding = self.tokenizer(caption, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")
            text_input_ids = encoding["input_ids"].squeeze(0)  # Remove batch dimension
            text_attention_mask = encoding["attention_mask"].squeeze(0)

            # Load Labels (Humor, Sarcasm, Offense, Motivation)
            labels = self.data.iloc[idx, 4:8].apply(lambda x: self.label_mapping.get(x, -1)).values # Map string labels to numerical values using label_mapping
            labels = torch.tensor(labels.astype(float), dtype=torch.float32) # Convert to float tensor after mapping to numerical values

            return text_input_ids, text_attention_mask, image, labels

        except (OSError, UnidentifiedImageError) as e:
            print(f"Error loading image {img_name}: {e}")
            # Skip this image and move to the next one
            # You can also choose to return default values or raise a custom exception
            text_input_ids = torch.zeros(self.max_length, dtype=torch.long)  # Assuming text_input_ids should have size max_length
            text_attention_mask = torch.zeros(self.max_length, dtype=torch.long) # Assuming text_attention_mask should have size max_length
            image = torch.zeros(3, 224, 224, dtype=torch.float)  # Assuming image should have size (3, 224, 224)
            labels = torch.zeros(4, dtype=torch.float32)  # Assuming labels should have size 4

            return text_input_ids, text_attention_mask, image, labels

In [ ]:
# Create Dataset & DataLoader
dataset = MemeDataset(CSV_FILE_PATH, IMG_DIR_PATH, tokenizer, transform=image_transform)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Check a sample batch
for text_input_ids, text_attention_mask, images, labels in dataloader:
    print("Text input shape:", text_input_ids.shape)  # (batch_size, max_length)
    print("Image shape:", images.shape)  # (batch_size, 3, 224, 224)
    print("Labels shape:", labels.shape)  # (batch_size, 4)
    break


Text input shape: torch.Size([8, 128])
Image shape: torch.Size([8, 3, 224, 224])
Labels shape: torch.Size([8, 4])


In [ ]:
# Load Pretrained Text Transformer (BERT or RoBERTa)
text_encoder = AutoModel.from_pretrained(TEXT_MODEL)

# Load Pretrained Image Model (ResNet50 or ViT)
IMAGE_MODEL = "resnet50"  # Change to "vit_base_patch16_224" for ViT
if IMAGE_MODEL == "resnet50":
    image_encoder = models.resnet50(pretrained=True)
    image_encoder.fc = nn.Identity()  # Remove classification head
elif IMAGE_MODEL == "vit_base_patch16_224":
    import timm
    image_encoder = timm.create_model("vit_base_patch16_224", pretrained=True)
    image_encoder.head = nn.Identity()  # Remove classification head


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
class MultimodalModel(nn.Module):
    def __init__(self, text_model, image_model, fusion_dim=512):
        super(MultimodalModel, self).__init__()
        self.text_encoder = text_model
        self.image_encoder = image_model

        text_embedding_dim = 768
        image_embedding_dim = 2048 if IMAGE_MODEL == "resnet50" else 768

        # Fusion layer
        self.fusion = nn.Linear(text_embedding_dim + image_embedding_dim, fusion_dim)
        self.relu = nn.ReLU()

        # Separate heads for each subtask
        self.humor_head = nn.Linear(fusion_dim, 4)  # 4 classes for humor
        self.sarcasm_head = nn.Linear(fusion_dim, 4)  # 4 classes for sarcasm
        self.offense_head = nn.Linear(fusion_dim, 4)  # 4 classes for offense
        self.motivation_head = nn.Linear(fusion_dim, 2)  # 2 classes for motivation

    def forward(self, text_input_ids, text_attention_mask, image_tensor):
        # Text encoding
        text_features = self.text_encoder(input_ids=text_input_ids, attention_mask=text_attention_mask).last_hidden_state[:, 0, :]

        # Image encoding
        image_features = self.image_encoder(image_tensor).squeeze()

        # Fusion
        fused_features = torch.cat((text_features, image_features), dim=1)
        fused_output = self.relu(self.fusion(fused_features))

        # Separate predictions for each subtask
        humor_output = self.humor_head(fused_output)
        sarcasm_output = self.sarcasm_head(fused_output)
        offense_output = self.offense_head(fused_output)
        motivation_output = self.motivation_head(fused_output)

        # Adding a dimension to the output tensors using unsqueeze
        return humor_output, sarcasm_output, offense_output, motivation_output

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = MultimodalModel(text_encoder, image_encoder).to(device)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)


In [ ]:
from sklearn.metrics import f1_score, accuracy_score

epochs = 500
for epoch in range(epochs):
    total_loss = 0
    all_preds = []
    all_labels = []

    model.train()

    for text_input_ids, text_attention_mask, images, labels in dataloader:
        text_input_ids, text_attention_mask, images, labels = (
            text_input_ids.to(device), text_attention_mask.to(device), images.to(device), labels.to(device)
        )

        optimizer.zero_grad()

        # Get predictions for each subtask
        humor_output, sarcasm_output, offense_output, motivation_output = model(text_input_ids, text_attention_mask, images)

        # Calculate loss for each subtask
        # Calculate loss for each subtask
        loss_humor = criterion(humor_output, labels[:, 0].unsqueeze(1).float())  # Convert labels to float and add dimension
        loss_sarcasm = criterion(sarcasm_output, labels[:, 1].unsqueeze(1).float())
        loss_offense = criterion(offense_output, labels[:, 2].unsqueeze(1).float())
        loss_motivation = criterion(motivation_output, labels[:, 3].unsqueeze(1).float())

        # Total loss
        loss = loss_humor + loss_sarcasm + loss_offense + loss_motivation

        loss.backward()
        optimizer.step()

        total_loss += loss.item()

        # Store predictions & true labels for metric calculation
        preds = torch.cat([humor_output, sarcasm_output, offense_output, motivation_output], dim=1)
        all_preds.append(preds.detach().cpu().numpy())
        all_labels.append(labels.cpu().numpy())

    # Convert to numpy arrays
    all_preds = np.vstack(all_preds)  # Shape: (num_samples, 4)
    all_labels = np.vstack(all_labels)  # Shape: (num_samples, 4)

    # Convert Regression Outputs to Discrete Labels (Round to Nearest Integer)
    all_preds = np.round(all_preds).astype(int)

    # Compute Metrics for Each Category
    f1_humor = f1_score(all_labels[:, 0], all_preds[:, 0], average="macro")
    f1_sarcasm = f1_score(all_labels[:, 1], all_preds[:, 1], average="macro")
    f1_offense = f1_score(all_labels[:, 2], all_preds[:, 2], average="macro")
    f1_motivation = f1_score(all_labels[:, 3], all_preds[:, 3], average="macro")

    acc_humor = accuracy_score(all_labels[:, 0], all_preds[:, 0])
    acc_sarcasm = accuracy_score(all_labels[:, 1], all_preds[:, 1])
    acc_offense = accuracy_score(all_labels[:, 2], all_preds[:, 2])
    acc_motivation = accuracy_score(all_labels[:, 3], all_preds[:, 3])

    avg_f1 = (f1_humor + f1_sarcasm + f1_offense + f1_motivation) / 4
    avg_acc = (acc_humor + acc_sarcasm + acc_offense + acc_motivation) / 4

    # Print Metrics
    print(f"Epoch [{epoch+1}/{epochs}] - Loss: {total_loss / len(dataloader):.4f}")
    print(f"  F1 Scores  - Humor: {f1_humor:.4f}, Sarcasm: {f1_sarcasm:.4f}, Offense: {f1_offense:.4f}, Motivation: {f1_motivation:.4f}, Avg: {avg_f1:.4f}")
    print(f"  Accuracy   - Humor: {acc_humor:.4f}, Sarcasm: {acc_sarcasm:.4f}, Offense: {acc_offense:.4f}, Motivation: {acc_motivation:.4f}, Avg: {avg_acc:.4f}")
    print("-" * 80)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Error loading image /content/images/images/image_5119.png: image file is truncated


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/5] - Loss: 2.4676
  F1 Scores  - Humor: 0.1886, Sarcasm: 0.2115, Offense: 0.1840, Motivation: 0.1708, Avg: 0.1887
  Accuracy   - Humor: 0.3566, Sarcasm: 0.4637, Offense: 0.3500, Motivation: 0.3028, Avg: 0.3682
--------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


...

Error loading image /content/images/images/image_5119.png: image file is truncated


/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [4/5] - Loss: 1.7957
  F1 Scores  - Humor: 0.3845, Sarcasm: 0.3289, Offense: 0.3091, Motivation: 0.1414, Avg: 0.2910
  Accuracy   - Humor: 0.4751, Sarcasm: 0.4604, Offense: 0.3914, Motivation: 0.2487, Avg: 0.3939
--------------------------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8, 4])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:608: UserWarning: Using a target size (torch.Size([8, 1])) that is different to the input size (torch.Size([8, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/PIL/Image.py:1045: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Error loading image /content/images/images/image_5119.png: image file is truncated
Epoch [5/5] - Loss: 1.5154
  F1 Scores  - Humor: 0.4069, Sarcasm: 0.2758, Offense: 0.2446, Motivation: 0.1279, Avg: 0.2638
  Accuracy   - Humor: 0.5559, Sarcasm: 0.4316, Offense: 0.3733, Motivation: 0.2637, Avg: 0.4061
--------------------------------------------------------------------------------
